<a href="https://colab.research.google.com/github/aml7hawaiiedu/CCAPLandCoverProject/blob/main/ccap_unet_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

This is Amanda's version of UNET_regression_demo_whp (Aron's original code)
  
This is an Earth Engine <> TenserFlow demonstration notebook. Suppose you want to predict a continuous output (regression) from a stack of continuous inputs. In this example, the output is impervious surface area from [NLCD](https://www.mrlc.gov/data) and the input is a Landsat 8 composite. The model is a [fully convolutional neural network  (FCNN)](https://www.cv-foundation.org/openaccess/content_cvpr_2015/papers/Long_Fully_Convolutional_Networks_2015_CVPR_paper.pdf), specifically [U-net](https://arxiv.org/abs/1505.04597). 

In [ ]:
# Google Cloud Platform authentication.
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Import Google Earth Engine Python API, authenticate user's credentials, and initialize the Earth Engine library.
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=9hpotAjVarQUTRs9qtJbDch_xKmc9w0z49QU7OUbd5o&tc=LOf0kZwEJdfhbMCNGFs__jDLgn2aBymhF1YQWXN8VPE&cc=DnE9n3IAhtZllkGvzK9IQMnNbHkY5Ykl44eClcMmTJM

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AVHEtk6I7NqmVlH_eANYaeIuAFTyJZWCqYlaFAyhVzHm5lcBbL2bx92465Y

Successfully saved authorization token.


In [ ]:
# Tensorflow is an open-source machine learning library developed by Google. It is wildly used for building deep learning models.
# Print the version.
import tensorflow as tf
print(tf.__version__)

2.12.0


In [ ]:
# Folium is a Python library used for creating interactive maps and visualizations.
# Print the version. 
import folium
print(folium.__version__)

0.14.0


In [ ]:
# Name of the Google Cloud Storage (GCS) bucket where data will be stored or accessed from.
# GCS is cloud-based object storage service used to store and access large amounts of data in a highly scalable and durable manner. 
# It is common to define a bucket which serves as a top-level container. 
BUCKET = 'remote_sensing_fuckit_bucket'
print(BUCKET)

remote_sensing_fuckit_bucket


In [ ]:
# Name of the folder that will be used to store data related to the wetland U-Net model.
# Name of the subfolder within 'FOLDER' directory that will be used to store training patches or data used to train the wetland classification model. 
# Name of the subfolder within 'FOLDER' directory that will be used to store evaluation patches or data used to test the wetland classification model. 
FOLDER = 'wetland_unet'
TRAINING_BASE = 'training_patches'
EVAL_BASE = 'eval_patches'

# These lines of code specify the input features for the wetland U-Net model and the response variable that the model will be trained to predict.
# opticalBands is assigned as a list of string values, which correspond to the names of the optical bands that are present in the Landsat satellite imagery.
# thermalBands is assigned a list of string values, which correspond to the names of the optical bands that are present in the Landsat satellite imagery.
# BANDS is assigned the concatenation of the 'opticalBands' and 'thermalBands'. This represents a list of all the bands that will be used as features in the U-Net model.
# RESPONSE is assigned the string value 'b1' which corresponds to the name of the response variable that the U-Net model will be trained to predict. 
# FEATURES is assigned the concatenation of the 'BANDS' list and the 'RESPONSE' variables. This represents a list of all the features that will be used to train the U-Net model. 
opticalBands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']
thermalBands = ['B10', 'B11']
BANDS = opticalBands + thermalBands
RESPONSE = 'b1'
FEATURES = BANDS + [RESPONSE]

# Specify the inputs
# These lines of code specify the size and shape of patches that the U-Net model expects as input, as well as a dictionary that maps featue names to columns in an TenserFlow dataset. 
# KERNEL_SIZE is assigned the integer value '128' which represents the size of patches that the U-Net model expects as input.
# KERNEL_SHAPE is assigned a list contning two elements, both of which are equal to 'KERNEL_SIZE'. This represents the shape of the patches the U-Net model expects as an input.
# COLUMNS is assigned a list of TenserFlow 'fixedLenFeature' objects, which specify the expected shape and data type of each feature column in a Tenserflow dataset. 
  # 'shape' is set to 'KERNEL_SHAPE' and 'dtype' is set to 'tf.float32'. The list comprehension used here creates one 'fixedLenFeature' object for each feature in the 'FEATURES' list/ 
# FEATURE_DICT is assigned a Python dictionary that maps feature names to their corresponding columns in a TenserFlow dataset. 
  # 'zip' is used to create tuples parining each feature name from the 'FEATURES' list with its corresponding 'fixedLenFeature' object from the 'COLUMNS' list. 
  # The 'dict' function is then used to convert this list of tuples into a dictionary. 
KERNEL_SIZE = 128
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]
COLUMNS = [
  tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for k in FEATURES
]
FEATURES_DICT = dict(zip(FEATURES, COLUMNS))

# Define the size of the training and evaluation datasets that will be used to train and evaluate the U-Net model. 
# TRAIN_SIZE is assigned the integer value '1600' which represents the size of the training dataset.
# EVAL_SIZE is assigned the integer value '800' which represents the size of the evaluation dataset. 
TRAIN_SIZE = 1600
EVAL_SIZE = 800

# Specify model training parameters.
# Define model training parameters that will be used to train and evaluate the U-Net model. 
# BATCH_SIZE represents the number of training examples that will be used in each training iteration. 
# EPOCHS represents the number of times that each training dataset will be used to train the model. 
# BUFFER_SIZE represents the number of elements from the training dataset that will be pre-fetched and buffered in order to optimize data reading during training. 
# OPTIMIZER specifies the Stochastic Gradient Descent optimization algorithm will be used during training. 
  # ? what other optimizers would work? Is this the best one for this type of model?
# LOSS specifies the Mean Squared Error Loss function will be used during training. 
# METRICS is asigned a list containing the sting value 'RootMeanSquareError' which specifies RMSE metric will be used to evaluate the performance of the model during training. 
BATCH_SIZE = 16
EPOCHS = 10
BUFFER_SIZE = 2000
OPTIMIZER = 'SGD'
LOSS = 'MeanSquaredError'
METRICS = ['RootMeanSquaredError']

print(RESPONSE)

b1


In [ ]:
# Use Landsat 8 surface reflectance data.
# Use EE Python API to initialize an image collection. 
# Specify Landsat 8 surface reflectance image collection. 
# l8sr is the name of the object to be used in subsequent operations.

l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')


# This code defines a function 'maskL8sr' that will be used to apply cloud masking to Landsat 8 surface reflectance images.
def maskL8sr(image):
  cloudShadowBitMask = ee.Number(2).pow(3).int()
  cloudsBitMask = ee.Number(2).pow(5).int()
  qa = image.select('pixel_qa')
  mask1 = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
    qa.bitwiseAnd(cloudsBitMask).eq(0))
  mask2 = image.mask().reduce('min')
  mask3 = image.select(opticalBands).gt(0).And(
          image.select(opticalBands).lt(10000)).reduce('min')
  mask = mask1.And(mask2).And(mask3)
  return image.select(opticalBands).divide(10000).addBands(
          image.select(thermalBands).divide(10).clamp(273.15, 373.15)
            .subtract(273.15).divide(100)).updateMask(mask)



# The image input data is a cloud-masked median composite.
# Filter the l8sr image collection by date, selecting only images from the date range identified. 
# Apply the mask function to each image in the resulting collection using the map() method. 
  # This function applies a mask to each image and rescales the reflectance and temperature bands to a 0-1 range. 
# Calculate the median value of the resulting image collection using the median() method.
# Assign the variable name; in this case 'ls_2017'. 
# The resulting 'ls_2017' image will be used for training and evaluation of our U-Net model. 

ls_2017 = l8sr.filterDate('2017-01-01', '2017-12-30').map(maskL8sr).median()


# Create single mosaic
#ls_3yr_mosaic =  ls_2017.addBands(ls_2018)
#ls_3yr_mosaic = ls_3yr_mosaic.addBands(ls_2019)

#Reset band names to new stack
#BANDS = ls_3yr_mosaic.bandNames().getInfo()
#FEATURES = BANDS + [RESPONSE]

# Use folium to visualize the imagery.
# map = folium.Map(location=[21.4, -158.])



# Use 'getMapID()' method to get a map ID for the 'ls_2017' image. 
# Specify that we want to visualize band 2 of the image using 'bands':['B2']. Set the min and max.
# Create a folium map object centered at specific location using 'folium.Map()'. 
# Add a tile layer to the map using 'folium.TileLayer()' method, specifying the URL format of the tiles using the 'tiles' argument and adding 'attr' attribution info. 
# Set the overlay flag to 'True' and provide a name for the layer using 'name'. 
# Finally, add the tile layer to the map using '.add_to()' method. 
  # This allows us to visualize the 'ls_2017' image in the folium map. 
mapid_17 =ls_2017.getMapId({'bands': ['B2'], 'min': 0, 'max': 0.3})
map = folium.Map(location=[21.4, -158.])
folium.TileLayer(
    tiles=mapid_17['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name = '17 median composite',
    ).add_to(map)

map

In [ ]:
# One Hot Encoding function.
# 'oneHot()' function is used for One Hot Encoding of an image. 
  # Each pixel in the image is replaced by a binary vector of length 'n' where 'n' is the number of classes in the image. 
# image : image to be encoded
# code : the class code that will be encoded as '1' in the output, while all other classes are encoded as '0'
# The function compares each pixel value in the input 'image' with the 'code' value using the 'ee.image.eq()' finction. 
# The 'ee.image.eq()' function returns a binary image with the same dimensions as the input image, where each pixel is '1' if the input pixel is equal to the given 'code' value and '0' otherwise. 
# The resulting binary image is returned as the 'one_hot_image' encoded. 
def oneHot(image,code = 1):
  one_hot_image = image.eq(code)
  return one_hot_image
  
print(oneHot)

<function oneHot at 0x7f124eec9ca0>


In [ ]:
# get imagecollection from google earth engine - NOAA C-CAP land cover data for Hawaii 
# This code gets an EE image collection which contains the land cover data for Hawaii form the NOAA Coastal Change Analysis Program (C-CAP)
wetland = ee.ImageCollection("projects/sat-io/open-datasets/HRLC/CCAP_HI_LC")

# Filter image collection to include images within a specified date range. 
# The 'mosaic()' function is used to combine all the images in the filtered collection into a single image, which represents the most recent pixel value at each location. 
# This is done to reduce the numer of images in the collection and to create a single image for analysis. 
wetland = wetland.filterDate('2010-01-01', '2022-01-01').mosaic()

# Mask out pixels with a value of 0 in the 'wetland' image using the mask() function.
# This ensures the modeld doesn't predict or train on data outside of the acutual land cover area. 
wetland = wetland.mask(wetland.gt(0))

# This generates a one-hot encoding representation of the land cover classification image for Hawaii obtained from the NOAA C-CAP dataset.
# For each of the 25 land cover classes present in the image, the code creates a binary image where pixels with the corresponding class are set to 1 and pixels with other classes are set to 0.
# The reutning binary iamges are stored in a list 'wetland_oneHot'.
# The list is converted to a single multi-band image.
# The bands of the multi-image are renamed to correspond to the class lables.
wetland_oneHot = []
for code in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25]:
  one_hot_image = oneHot(wetland,code)
  wetland_oneHot.append(one_hot_image)
wetland_oneHot = ee.Image(wetland_oneHot)
wetland_oneHot = wetland_oneHot.rename(['p1','p2','p3','p4','p5','p6','p7','p8','p9','p10','p11','p12','p13','p14','p15','p16','p17','p18','p19','p20','p21','p22','p23','p24','p25'])

# Use folium to visualize the imagery.
# The 'wetland_oneHot' image is used to generate the map ID. 
# The min and max are set to 0 and 1 to display the binary values of the one-hot encoding. 
mapid = wetland_oneHot.getMapId({'bands': ['p15','p10','p2'],'min': 0, 'max': 1})
map = folium.Map(location=[21.4, -158])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='NOAA CCAP Land Cover',
  ).add_to(map)
map.add_child(folium.LayerControl())
map







Stack the 2D images (Landsat  and NOAA C-CAP land cover data for Hawaii) to create a single image from which samples can be taken. 

Convert the image into an array image in which each pixel stores 128x128 patches of pixels for each band. 

This is a key step that bears emphasis: to export training patches, convert a multi-band image to an array image using neighborhoodToArray(), then sample the image at points.

In [ ]:
# There are 25 strings in the list. 
RESPONSE = ['p1','p2','p3','p4','p5','p6','p7','p8','p9','p10','p11','p12','p13','p14','p15','p16','p17','p18','p19','p20','p21','p22','p23','p24','p25']
print(RESPONSE)
print(BANDS)


# Create a feature stack by concatenating two image collections and converting the result to float.
featureStack = ee.Image.cat([
  ls_2017.select(BANDS),
  wetland_oneHot.select(RESPONSE)
]).float()

# Define a kernel that will be used for image processing operations. 
# The kernel is initialized with a list of ones, meaning that each pixel in the kernel will have a weight of one when applied to the image. 
list = ee.List.repeat(1, KERNEL_SIZE)
lists = ee.List.repeat(list, KERNEL_SIZE)
kernel = ee.Kernel.fixed(KERNEL_SIZE, KERNEL_SIZE, lists)

# Convert the neighborhood values within the kernel to arrays.
# Create a new array image where each pixel value is an array representing the neighborhood of the corresponding pixel in the original feature stack. 
# THe size of the neighborhood is dertermined by the size of the kernel. 
# 'neighborhoodToArray' is an EE function that converst an image to an array image. 
arrays = featureStack.neighborhoodToArray(kernel)

['p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9', 'p10', 'p11', 'p12', 'p13', 'p14', 'p15', 'p16', 'p17', 'p18', 'p19', 'p20', 'p21', 'p22', 'p23', 'p24', 'p25']
['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11']


In [ ]:
# RESPONSE = 'b1'

FEATURES = BANDS + RESPONSE


# Specify the size and shape of patches expected by the model.
# KERNEL_SIZE = KERNEL_SIZE
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]
COLUMNS = [
  tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for k in FEATURES
]
FEATURES_DICT = dict(zip(FEATURES, COLUMNS))

print(FEATURES_DICT)


{'B1': FixedLenFeature(shape=[128, 128], dtype=tf.float32, default_value=None), 'B2': FixedLenFeature(shape=[128, 128], dtype=tf.float32, default_value=None), 'B3': FixedLenFeature(shape=[128, 128], dtype=tf.float32, default_value=None), 'B4': FixedLenFeature(shape=[128, 128], dtype=tf.float32, default_value=None), 'B5': FixedLenFeature(shape=[128, 128], dtype=tf.float32, default_value=None), 'B6': FixedLenFeature(shape=[128, 128], dtype=tf.float32, default_value=None), 'B7': FixedLenFeature(shape=[128, 128], dtype=tf.float32, default_value=None), 'B10': FixedLenFeature(shape=[128, 128], dtype=tf.float32, default_value=None), 'B11': FixedLenFeature(shape=[128, 128], dtype=tf.float32, default_value=None), 'p1': FixedLenFeature(shape=[128, 128], dtype=tf.float32, default_value=None), 'p2': FixedLenFeature(shape=[128, 128], dtype=tf.float32, default_value=None), 'p3': FixedLenFeature(shape=[128, 128], dtype=tf.float32, default_value=None), 'p4': FixedLenFeature(shape=[128, 128], dtype=tf.

In [ ]:
print(featureStack.bandNames().getInfo())
# print(arrays)

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9', 'p10', 'p11', 'p12', 'p13', 'p14', 'p15', 'p16', 'p17', 'p18', 'p19', 'p20', 'p21', 'p22', 'p23', 'p24', 'p25']


Use some pre-made geometries to sample the stack in strategic locations. 

Specifically, these are hand-made polygons in which to take the 128x128 samples. 

Display the sampling polygons on a map, red for training polygons, blue for evaluation.

In [ ]:
# FeatureCollection is a colelction of featuers, where each feature is represented as a dictionary of properties and a geometry that describes its spatial extent. 
# The trainingPolys features in this are used to train the model to recognize land cover types based on the spectral and spatial features extracted from satellite imagery. 
# The evalPolys features are used to evaluate the accuracy of the trained model. 

# trainingPolys = ee.FeatureCollection('projects/ee-seismosmsr-landcover/assets/trainingPolys_ccap')
# evalPolys = ee.FeatureCollection('projects/ee-seismosmsr-landcover/assets/evalPolys')

trainingPolys = ee.FeatureCollection('projects/ee-aml7-001/assets/trainingPolys') 
evalPolys = ee.FeatureCollection('projects/ee-aml7-001/assets/evalPolys')

# Create an empty image.
# Create a labled image where the pixles belonging to the training set have a value of 1 and evaluation set have a value of 2.
polyImage = ee.Image(0).byte().paint(trainingPolys, 1).paint(evalPolys, 2)
polyImage = polyImage.updateMask(polyImage)


# Use folium to visualize EE image. 

mapid = polyImage.getMapId({'min': 1, 'max': 2, 'palette': ['red', 'blue']})
map = folium.Map(location=[21.4, -158.], zoom_start=4)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='training polygons',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

In [ ]:
# Convert the feature collections to lists for iteration.
trainingPolysList = trainingPolys.toList(trainingPolys.size())
evalPolysList = evalPolys.toList(evalPolys.size())

# These numbers determined experimentally.
n = 5 # Number of shards in each polygon.
N = 200 # Total sample size in each polygon.

# Export all the training data (in many pieces), with one task 
# per geometry.
print(BANDS+RESPONSE)
# print()
for g in range(trainingPolys.size().getInfo()):
  geomSample = ee.FeatureCollection([])
  for i in range(n):
    sample = arrays.sample(
      region = ee.Feature(trainingPolysList.get(g)).geometry(), 
      scale = 30,
      numPixels = N / n, # Size of the shard.
      seed = i,
      tileScale = 8
    )
    geomSample = geomSample.merge(sample)
    
  desc = TRAINING_BASE + '_g' + str(g)
  task = ee.batch.Export.table.toCloudStorage(
    collection = geomSample,
    description = desc,
    bucket = BUCKET,
    fileNamePrefix = FOLDER + '/' + desc,
    fileFormat = 'TFRecord'
    # selectors = BANDS + RESPONSE
  )
  task.start()

# Export all the evaluation data.
for g in range(evalPolys.size().getInfo()):
  geomSample = ee.FeatureCollection([])
  for i in range(n):
    sample = arrays.sample(
      region = ee.Feature(evalPolysList.get(g)).geometry(), 
      scale = 30,
      numPixels = N / n,
      seed = i,
      tileScale = 8
    )
    geomSample = geomSample.merge(sample)

  desc = EVAL_BASE + '_g' + str(g)
  task = ee.batch.Export.table.toCloudStorage(
    collection = geomSample,
    description = desc,
    bucket = BUCKET,
    fileNamePrefix = FOLDER + '/' + desc,
    fileFormat = 'TFRecord'
    # selectors = BANDS + RESPONSE
  )
  task.start()

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9', 'p10', 'p11', 'p12', 'p13', 'p14', 'p15', 'p16', 'p17', 'p18', 'p19', 'p20', 'p21', 'p22', 'p23', 'p24', 'p25']


In [ ]:

def parse_tfrecord(example_proto):
  """The parsing function.
  Read a serialized example into the structure defined by FEATURES_DICT.
  Args:
    example_proto: a serialized Example.
  Returns:
    A dictionary of tensors, keyed by feature name.
  """
  return tf.io.parse_single_example(example_proto, FEATURES_DICT)


def to_tuple(inputs):
  """Function to convert a dictionary of tensors to a tuple of (inputs, outputs).
  Turn the tensors returned by parse_tfrecord into a stack in HWC shape.
  Args:
    inputs: A dictionary of tensors, keyed by feature name.
  Returns:
    A tuple of (inputs, outputs).
  """
  inputsList = [inputs.get(key) for key in FEATURES]
  stacked = tf.stack(inputsList, axis=0)
  # Convert from CHW to HWC
  stacked = tf.transpose(stacked, [1, 2, 0])
  return stacked[:,:,:len(BANDS)], stacked[:,:,len(BANDS):]


def get_dataset(pattern):
  """Function to read, parse and format to tuple a set of input tfrecord files.
  Get all the files matching the pattern, parse and convert to tuple.
  Args:
    pattern: A file pattern to match in a Cloud Storage bucket.
  Returns:
    A tf.data.Dataset
  """
  glob = tf.io.gfile.glob(pattern)
  dataset = tf.data.TFRecordDataset(glob, compression_type='GZIP')
  dataset = dataset.map(parse_tfrecord, num_parallel_calls=5)
  dataset = dataset.map(to_tuple, num_parallel_calls=5)
  return dataset


print(get_dataset)

<function get_dataset at 0x7f124ed11550>


In [ ]:
# Define a function that returns a preprocessed dataset for ML. 
# Match the training data files in GCS bucket.
# Calls 'get_dataset' function to read, parse, and convert the input files to tuples of tensors. 
# The resulting dataset is shuffled, batched, and repeated (epochs) to prepare for ML training. 
# Save the resulting dataset to 'training' variable. 
def get_training_dataset():
	"""Get the preprocessed training dataset
  Returns: 
    A tf.data.Dataset of training data.
  """
	glob = 'gs://' + BUCKET + '/' + FOLDER + '/' + TRAINING_BASE + '*'
	# print(glob)
	dataset = get_dataset(glob)
	dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
	return dataset

training = get_training_dataset()

# print(iter(training.take(2)).next())
# This appears to be a tensor with shape (16, 128, 128, 9) and data type float32. 
# The first dimension has a size of 16, which suggests that this tensor contains a batch of 16 samples. 
# The next two dimensions have a size of 128, which suggests that each sample has an image with a resolution of 128 x 128 pixels. 
# The last dimension has a size of 9, which suggests that each pixel is represented by a 9-dimensional feature vector. 

(<tf.Tensor: shape=(16, 128, 128, 9), dtype=float32, numpy=
array([[[[0.0331 , 0.0419 , 0.0736 , ..., 0.1453 , 0.3165 , 0.2705 ],
         [0.0404 , 0.0467 , 0.077  , ..., 0.1657 , 0.3155 , 0.2705 ],
         [0.0431 , 0.0478 , 0.0762 , ..., 0.1799 , 0.3175 , 0.2715 ],
         ...,
         [0.0338 , 0.0424 , 0.0813 , ..., 0.121  , 0.2835 , 0.2465 ],
         [0.0339 , 0.0414 , 0.0798 , ..., 0.1174 , 0.2855 , 0.2475 ],
         [0.0304 , 0.0385 , 0.06845, ..., 0.1148 , 0.2865 , 0.2555 ]],

        [[0.0329 , 0.0427 , 0.0825 , ..., 0.1379 , 0.3135 , 0.2665 ],
         [0.0358 , 0.0427 , 0.0764 , ..., 0.1515 , 0.3135 , 0.2665 ],
         [0.042  , 0.048  , 0.0762 , ..., 0.1758 , 0.3135 , 0.2695 ],
         ...,
         [0.03945, 0.05255, 0.10395, ..., 0.14705, 0.2895 , 0.2575 ],
         [0.0396 , 0.0527 , 0.1057 , ..., 0.1431 , 0.2895 , 0.2525 ],
         [0.0378 , 0.0491 , 0.0943 , ..., 0.1403 , 0.292  , 0.2605 ]],

        [[0.0351 , 0.0428 , 0.0822 , ..., 0.1348 , 0.3105 , 0.2635 ]

In [ ]:
# Return a Tenserflow dataset contaiing the preprocessed evaluation data. 
# File path to the evaluation data stored in GCS bucket.
# Calls 'get_dataset' function to create a TenserFlow dataset containing the evaluation data. 
# Evaluation dataset is batched with a batchsize of 1, meaning each element of the dataset consists of a single example. 
# The function returns the resulting evaluation dataset.
def get_eval_dataset():
	"""Get the preprocessed evaluation dataset
  Returns: 
    A tf.data.Dataset of evaluation data.
  """
	glob = 'gs://' + BUCKET + '/' + FOLDER + '/' + EVAL_BASE + '*'
	print(glob)
	dataset = get_dataset(glob)
	dataset = dataset.batch(1).repeat()
	return dataset

evaluation = get_eval_dataset()

gs://remote_sensing_fuckit_bucket/wetland_unet/eval_patches*


In [ ]:
evaluation

<_RepeatDataset element_spec=(TensorSpec(shape=(None, 128, 128, 9), dtype=tf.float32, name=None), TensorSpec(shape=(None, 128, 128, 25), dtype=tf.float32, name=None))>

In [ ]:
print(tf.__version__)

2.12.0


In [ ]:
# import TenserFlow classes and functions 
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import models
from tensorflow.keras import metrics
from tensorflow.keras import optimizers
from tensorflow.keras.layers import BatchNormalization

# from tensorflow.python.keras.layers.normalization import BatchNormalization
# from tensorflow.python.keras.layers.normalization import BatchNormalization

# from tensorflow.keras.layers import BatchNormalization
# from keras.layers.normalization.batch_normalization import BatchNormalization
# from tensorflow.python.keras.layers import BatchNormalization 

# U-Net model for image segmentation. 
# Encoder and decoder conncted by a center block. 
# Encoder downsamples the input image while capturing its features. 
# Decoder upsamples the encoded image to generate a segmentation map.
def conv_block(input_tensor, num_filters):
	encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(input_tensor)
	encoder = layers.BatchNormalization()(encoder)
	encoder = layers.Activation('relu')(encoder)
	encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(encoder)
	encoder = layers.BatchNormalization()(encoder)
	encoder = layers.Activation('relu')(encoder)
	return encoder

def encoder_block(input_tensor, num_filters):
	encoder = conv_block(input_tensor, num_filters)
	encoder_pool = layers.MaxPooling2D((2, 2), strides=(2, 2))(encoder)
	return encoder_pool, encoder

def decoder_block(input_tensor, concat_tensor, num_filters):
	decoder = layers.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(input_tensor)
	decoder = layers.concatenate([concat_tensor, decoder], axis=-1)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	return decoder

def get_model():
	inputs = layers.Input(shape=[KERNEL_SIZE, KERNEL_SIZE, len(BANDS)]) # 256
	encoder0_pool, encoder0 = encoder_block(inputs, 32) # 128
	encoder1_pool, encoder1 = encoder_block(encoder0_pool, 64) # 64
	encoder2_pool, encoder2 = encoder_block(encoder1_pool, 128) # 32
	encoder3_pool, encoder3 = encoder_block(encoder2_pool, 256) # 16
	encoder4_pool, encoder4 = encoder_block(encoder3_pool, 512) # 8
	center = conv_block(encoder4_pool, 1024) # center
	decoder4 = decoder_block(center, encoder4, 512) # 16
	decoder3 = decoder_block(decoder4, encoder3, 256) # 32
	decoder2 = decoder_block(decoder3, encoder2, 128) # 64
	decoder1 = decoder_block(decoder2, encoder1, 64) # 128
	decoder0 = decoder_block(decoder1, encoder0, 32) # 256
	outputs = layers.Conv2D(25, (1, 1), activation='softmax')(decoder0)

	model = models.Model(inputs=[inputs], outputs=[outputs])

	model.compile(
		optimizer=optimizers.get(OPTIMIZER), 
		loss=losses.get(LOSS),
		metrics=[metrics.get(metric) for metric in METRICS])

	return model

In [ ]:
m = get_model()
print(m.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 9  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 128, 128, 32  2624        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 128, 128, 32  128        ['conv2d[0][0]']                 
 alization)                     )                                                             

In [ ]:
# m = get_model()
# tf.keras.utils.plot_model(m)
# Load a trained model. 50 epochs. 25 hours. Final RMSE ~0.08.
MODEL_DIR = 'gs://' + BUCKET +'/'+ FOLDER +  '/'

m = tf.keras.models.load_model(MODEL_DIR)
# EPOCHS
#int(TRAIN_SIZE / BATCH_SIZE)
# m.fit(
#     x=training, 
#     epochs=100, 
#     steps_per_epoch=20, 
#     validation_data=evaluation,
#     validation_steps=EVAL_SIZE)

# m.save('gs://' + BUCKET +'/'+ FOLDER +  '/')
print(m)